In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig
import configs as c
from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatCompletion

from semantic_kernel.connectors.ai.google.google_ai.google_ai_prompt_execution_settings import (
    GoogleAIPromptExecutionSettings,
)

from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory

In [50]:
# Initialize the kernel
kernel = Kernel()

history = ChatHistory()

In [51]:
# chat_completion = OllamaChatCompletion(
#         ai_model_id='llama3.2'
#     )

# kernel.add_service(chat_completion)

In [52]:
from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatCompletion

chat_completion= GoogleAIChatCompletion(
    gemini_model_id="gemini-2.0-flash",
    api_key=c.GOOGLE_API_KEY,
)

kernel.add_service(chat_completion)

In [53]:
execution_settings = GoogleAIPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()


In [59]:
user_input = "What is the capital of Sri Lanka?"

In [60]:
history.add_user_message(user_input)

In [61]:
result = await chat_completion.get_chat_message_content(
            settings=execution_settings,
            chat_history=history,
            kernel=kernel,
        )
print(result)

history.add_message(result)

Sri Lanka has two capitals:

*   **Sri Jayawardenepura Kotte** is the administrative capital and the location of the parliament.
*   **Colombo** is the commercial capital and largest city.



## Creating a MCP Server from the Kernel

In [62]:


@kernel_function()
def echo_function(message: str, extra: str = "") -> str:
    """Echo a message as a function"""
    return f"Function echo: {message} {extra}"


kernel.add_function("echo", echo_function, "echo_function")
kernel.add_function(
    plugin_name="prompt",
    function_name="prompt",
    prompt_template_config=PromptTemplateConfig(
        name="prompt",
        description="This is a prompt",
        template="Please repeat this: {{$message}} and this: {{$extra}}",
        input_variables=[
            InputVariable(
                name="message",
                description="This is the message.",
                is_required=True,
                json_schema='{ "type": "string", "description": "This is the message."}',
            ),
            InputVariable(
                name="extra",
                description="This is extra.",
                default="default",
                is_required=False,
                json_schema='{ "type": "string", "description": "This is the message."}',
            ),
        ],
    ),
)
server = kernel.as_mcp_server(server_name="sk")